In [ ]:
import os
import random
import numpy as np
import pandas as pd

from dataclasses import dataclass
from typing import List, Tuple, Union
from PIL import Image

import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import cv2

from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from sklearn.model_selection import StratifiedGroupKFold
from tqdm import tqdm 

In [ ]:
def system_config (SEED_VALUE=42, package_list=None):

    random.seed(SEED_VALUE)
    np.random.seed(SEED_VALUE)
    torch.manual_seed(SEED_VALUE)

    def is_running_in_kaggle():
        return "KAGGLE_KERNEL_RUN_TYPE" in os.environ
    
    #GPU check
    if torch.cuda.is_available():
        print("Using CUDA GPU")


        
        DEVICE = torch.device("cuda")
        print("Devide: ",  DEVICE)
        GPU_AVAILABLE = True

        torch.cuda.manual_seed_all(SEED_VALUE)
        torch.cuda.manual_seed(SEED_VALUE)

        torch.backends.cudnn.enabled = True       # Provides highly optimized primitives for DL operations.
        torch.backends.cudnn.deterministic = True # Insures deterministic even when above cudnn is enabled.
        torch.backends.cudnn.benchmark = False    # Setting to True can cause non-deterministic behavior.

    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        print('Using Apple Silicon GPU')

        # Set the device to the Apple Silicon GPU Metal Performance Shader (MPS).
        DEVICE = torch.device("mps")
        print("Device: ", DEVICE)
        # Environment variable that allows PyTorch to fall back to CPU execution
        # when encountering operations that are not currently supported by MPS.
        os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
        GPU_AVAILABLE = True

        torch.mps.manual_seed(SEED_VALUE)
        torch.use_deterministic_algorithms(True)

    else:
        print('Using CPU')
        DEVICE = torch.device('cpu')
        print("Device: ", DEVICE)
        GPU_AVAILABLE = False

        torch.use_deterministic_algorithms(True)

    return str(DEVICE), GPU_AVAILABLE

DEVICE, GPU_AVAILABLE = system_config()

In [ ]:
# entorn variables
RANDOM_SEED = 42
DATA_DIR = os.path.join("..", "data")
LABEL_WEIGHTS = {
    "Dry_Green_g": 0.1,
    "Dry_Dead_g": 0.1,
    "Dry_Clover_g": 0.1,
    "GDM_g": 0.2,
    "Dry_Total_g": 0.5
}
N_SPLITS = 3
TARGET_NAMES= list(LABEL_WEIGHTS.keys())

In [ ]:
#implement competition loss function 
def globally_weighted_r_squared(y_true: torch.tensor, y_pred: torch.tensor, target_class: list[str], eps: float = 1e-12) -> torch.tensor:
    y_true = y_true.view(-1).float()
    y_pred = y_pred.view(-1).float()

    if y_true.shape != y_pred.shape:
        raise ValueError(f"Shapes must match. Got {y_true.shape} vs {y_pred.shape}.")

    weights = torch.tensor([LABEL_WEIGHTS[cls] for cls in target_class], device=y_true.device, dtype=y_true.dtype)

    y_bar = torch.sum(weights * y_true)/weights.sum()
    ss_res = torch.sum(weights * (y_true - y_pred) ** 2)
    ss_tot = torch.sum(weights * (y_true - y_bar) ** 2) + eps

    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

# Plotting and diagnostics
def plot_training_curves(
    train_losses, val_losses,
    train_metric, val_metric,
    metric_name="R²"
):
    epochs = range(1, len(train_losses) + 1)

    # --- LOSS ---
    plt.figure(figsize=(10, 4))
    plt.plot(epochs, train_losses, label="Train Loss")
    plt.plot(epochs, val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE)")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # --- METRIC ---
    plt.figure(figsize=(10, 4))
    plt.plot(epochs, train_metric, label=f"Train {metric_name}")
    plt.plot(epochs, val_metric, label=f"Val {metric_name}")
    plt.xlabel("Epoch")
    plt.ylabel(metric_name)
    plt.title(f"Training vs Validation {metric_name}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

#Image Preprocessing

# def resize_split(
#         src_dir:str,
#         dst_dir:str,
#         size_hw=(256, 256),
#         verbose:bool=True
#         ):
#     """
#     Resize all images in src_dir and save into dst_dir.
#     """
#     os.makedirs(dst_dir, exist_ok=True)
#     H, W = size_hw
#     files = [f for f in os.listdir(src_dir) if not f.startswith('.')]

#     n_ok, n_fail = 0, 0

#     for file in tqdm(files, disable=not verbose):
#         in_path = os.path.join(src_dir, file)
#         out_path = os.path.join(dst_dir, file)

#         img = cv2.imread(in_path, cv2.IMREAD_COLOR)
#         if img is None:
#             n_fail += 1
#             continue
#         resized = cv2.resize(img, (W, H), interpolation=cv2.INTER_AREA)
#         ok = cv2.imwrite(out_path, resized)
#         if ok:
#             n_ok += 1
#         else:
#             n_fail += 1
#     print(f"✅ Done. OK: {n_ok} | Failed: {n_fail}")
    

In [ ]:
# RAW_DIR = os.path.join("..", "data", "raw")
# RESIZED_DIR = os.path.join("..", "data", "resized")

# resize_split(
#     src_dir=os.path.join(RAW_DIR, "train"),
#     dst_dir=os.path.join(RESIZED_DIR, "train"),
#     size_hw=(128, 64),
# )

# resize_split(
#     src_dir=os.path.join(RAW_DIR, "test"),
#     dst_dir=os.path.join(RESIZED_DIR, "test"),
#     size_hw=(128, 64),
# )


In [ ]:
#TODO: implement robust validation set
train_df = pd.read_csv(os.path.join(DATA_DIR, "raw", "train.csv"))
train_df["id"] = train_df["image_path"].apply(lambda row: row.replace("train/", "").replace(".jpg", ""))
train_df_ids = train_df.groupby("id", as_index=False).agg(
    Pre_GSHH_NDVI = ("Pre_GSHH_NDVI", "first"),
    Height_Ave_cm = ("Height_Ave_cm", "first"),
)

train_df_ids["ndvi_bin"] = pd.qcut(train_df_ids["Pre_GSHH_NDVI"], q=3, duplicates="drop")
train_df_ids["h_bin"] = pd.qcut(train_df_ids["Height_Ave_cm"], q=3, duplicates="drop")

train_df_ids["stratify_group"] = train_df_ids["ndvi_bin"].astype(str) + "_" + train_df_ids["h_bin"].astype(str)

sgkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state= RANDOM_SEED)

train_df_ids["fold"] = -1

X = train_df_ids[["Pre_GSHH_NDVI", "Height_Ave_cm"]]
y = train_df_ids["stratify_group"]
groups = train_df_ids["id"]

for fold, (_, val_idx) in enumerate(sgkf.split(X, y, groups=groups)):
    train_df_ids.loc[val_idx, "fold"] = fold

train_df = train_df.merge(train_df_ids[["id", "fold"]], on="id", how="left")
assert train_df["fold"].isna().sum() == 0

check = train_df_ids.groupby("fold")[["Pre_GSHH_NDVI", "Height_Ave_cm"]].describe()
check

In [ ]:
train_df.head()

In [ ]:
FOLD = 0 #Choose validation fold

train_df_fold = train_df[train_df["fold"] != FOLD].reset_index(drop=True)
val_df_fold = train_df[train_df["fold"] == FOLD].reset_index(drop=True)

assert set(train_df_fold["id"]).isdisjoint(set(val_df_fold["id"]))

images_df_train = (
    train_df_fold[["id", "image_path"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

def build_targets(df):
    images = df[["id", "image_path"]].drop_duplicates()
    targets = df.pivot(index="id", columns="target_name", values="target").reset_index()
    target_df = images.merge(targets, on="id") 

    return target_df

train_targets = build_targets(train_df_fold)
val_targets = build_targets(val_df_fold)

train_targets.head()

In [ ]:
train_targets[TARGET_NAMES] = train_targets[TARGET_NAMES].apply(pd.to_numeric, errors="coerce")
val_targets[TARGET_NAMES]   = val_targets[TARGET_NAMES].apply(pd.to_numeric, errors="coerce")

In [ ]:
class BiomassDataset(Dataset):
    def __init__(self, df, image_root, transform=None):
        self.df = df
        self.image_root = image_root
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_path = os.path.join(self.image_root, row["image_path"])
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(img_path)
        image = image[:, :, ::-1] #Take all channels
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        y =  torch.from_numpy(
            row[TARGET_NAMES].astype("float32").values,
        )
        return image, y

class BiomassTestDataset(Dataset):
    def __init__(self, df_unique, image_root, transform=None):
        self.df = df_unique.reset_index(drop=True)
        self.image_root = image_root
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_root, row["image_path"])

        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if image is None:
            raise FileNotFoundError(img_path)

        green = image[:, :, ::-1]               # (H,W,3)
        pil_img = Image.fromarray(green)     # PIL

        if self.transform:
            x = self.transform(pil_img)      # tensor (1,H,W)
        else:
            x = transforms.ToTensor()(pil_img)

        return x, row["image_path"]

In [ ]:
print(TARGET_NAMES)
print([c for c in TARGET_NAMES if c not in train_targets.columns])

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(256),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(degrees=15),  
    transforms.RandomApply([transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.02)], p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:
train_ds = BiomassDataset(
    train_targets, 
    image_root=os.path.join(DATA_DIR, "raw"),
    transform=train_transforms
)

val_ds = BiomassDataset(
    val_targets,
    image_root=os.path.join(DATA_DIR, "raw"),
    transform=val_transforms
)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)


In [ ]:
def expand_preds_3_to_5(pred3: torch.Tensor) -> torch.Tensor:
    # pred3: (B, 3) -> [Green, Dead, Clover]
    green = pred3[:, 0:1]
    dead = pred3[:, 1:2]
    clover = pred3[:, 2:3]

    gdm = green + clover
    total = green + dead + clover
    pred5 = torch.cat([green, dead, clover, gdm, total], dim=1)  # (B, 5)
    return pred5

class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.n_outputs = len(TARGET_NAMES)

        self.backbone = nn.Sequential( #in 256,256) 
            nn.Conv2d(3, 6, kernel_size=5), #After conv ((256+2p-k)/s)+1 p =0 y s =1 252 shape after = (252, 252, 6)
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2), # (126, 126, 6)

            nn.Conv2d(6, 16, kernel_size=5), # (122, 122, 16)
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2), # (61, 61)
        )
        self.head = self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 61 * 61, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, self.n_outputs-2),
            nn.Softplus()
        )
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x

def train(
    DEVICE: torch.device,
    model: nn.Module,
    optimizer: optim.Optimizer,
    train_loader: DataLoader,
    epoch_index: int
) -> Tuple[float, float]:

    model.train()

    step_loss = 0.0
    n_samples = 0

    all_y_true, all_y_pred, all_target_class = [], [], []

    criterion = nn.SmoothL1Loss(beta=5.0)

    for images, targets in train_loader:
        images = images.to(DEVICE)
        targets = targets.to(DEVICE).float()

        optimizer.zero_grad()

        outputs_3 = model(images)
        loss = criterion(outputs_3, targets[:, :3])

        loss.backward()
        optimizer.step()

        batch_size = images.size(0)
        step_loss += loss.item() * batch_size
        n_samples += batch_size

        outputs_5 = expand_preds_3_to_5(outputs_3)

        B = targets.size(0)
        all_y_true.append(targets.detach().cpu().reshape(-1))
        all_y_pred.append(outputs_5.detach().cpu().reshape(-1))
        all_target_class.extend(TARGET_NAMES * B)

    epoch_loss = step_loss / max(n_samples,1)
    y_true = torch.cat(all_y_true)
    y_pred = torch.cat(all_y_pred)
    epoch_r2 = globally_weighted_r_squared(y_true, y_pred, all_target_class).item()

    print(f"Epoch {epoch_index}: Train loss={epoch_loss:.4f}, R^2={epoch_r2:.4f}")
    return epoch_loss, epoch_r2


def validate(DEVICE: torch.device,
             model: nn.Module,
             val_loader: DataLoader,
             epoch_index: int
            ) -> Tuple[float, float]:
    
    model.eval() #Set model to evaluation mode

    step_loss = 0.0
    n_samples = 0

    criterion = nn.SmoothL1Loss(beta=5.0)

    all_y_true = []
    all_y_pred = []
    all_target_class = []

    for data, target in val_loader:
        data = data.to(DEVICE)
        target = target.to(DEVICE).float()

        with torch.no_grad():
            outputs_3 = model(data)
            loss = criterion(outputs_3, target[:, :3])

        batch_size = data.size(0)
        step_loss += loss.item() * batch_size
        n_samples += batch_size 

        outputs_5 = expand_preds_3_to_5(outputs_3)
        B = target.size(0)
        all_y_true.append(target.detach().cpu().reshape(-1))
        all_y_pred.append(outputs_5.detach().cpu().reshape(-1))
        all_target_class.extend(TARGET_NAMES * B)
    
    val_loss = step_loss/max(n_samples,1)
    y_true = torch.cat(all_y_true)
    y_pred = torch.cat(all_y_pred)
    val_r2 = globally_weighted_r_squared(y_true, y_pred, all_target_class).item()
    print(f"Epoch {epoch_index}: Val loss={val_loss:.4f}, R^2={val_r2:.4f}")
    return val_loss, val_r2

    

In [ ]:
xb, yb = next(iter(train_loader))
print(xb.shape, yb.shape) 

In [ ]:
model = Regressor()
model.to(DEVICE)

In [ ]:
N_EPOCHS = 200

train_losses, train_comp_metric = [], []
val_losses, val_comp_metric = [], []

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode="max", factor=0.5, patience=10, min_lr = 1e-6
)

best_val_r2 = -np.inf
best_epoch = -1
best_path = "best_model.pt"

patience = 20
wait = 0

for epoch in range(1, N_EPOCHS + 1):
    train_loss, train_r2 = train(DEVICE, model, optimizer, train_loader, epoch)
    val_loss, val_r2     = validate(DEVICE, model, val_loader, epoch)
    scheduler.step(val_r2)

    train_losses.append(train_loss)
    train_comp_metric.append(train_r2)
    val_losses.append(val_loss)
    val_comp_metric.append(val_r2)

    if val_r2 > best_val_r2:
        wait = 0
        best_val_r2 = val_r2
        best_epoch = epoch
        torch.save(model.state_dict(), best_path)
        print(f"  + New best model saved at epoch {best_epoch} with Val R^2={best_val_r2:.4f}")
    else:
        wait += 1
        if wait >= patience:
            print(f"Early stopping at epoch {epoch}. Best epoch was {best_epoch} with Val R^2={best_val_r2:.4f}")
            break

    print("Best Epoch:", best_epoch, "Best Val R^2:", best_val_r2)

In [ ]:
plot_training_curves(
    train_losses=train_losses,
    val_losses=val_losses,
    train_metric=train_comp_metric,
    val_metric=val_comp_metric,
    metric_name="Weighted R²"
)

In [ ]:
best_model = Regressor().to(DEVICE)
best_model.load_state_dict(torch.load(best_path, map_location=DEVICE))

In [ ]:
def make_submission(
    model: torch.nn.Module,
    device: torch.device,
    test_df_long: pd.DataFrame,     # columns: sample_id, image_path, target_name
    image_root: str,                # carpeta donde están las imágenes resized (ej ../data/resized)
    transform,
    out_csv_path: str = "submission.csv",
    batch_size: int = 32,
):
    model.eval()
    model.to(device)

    # 1) una fila por imagen
    df_unique = test_df_long[["image_path"]].drop_duplicates().reset_index(drop=True)

    # 2) dataset + loader
    test_ds = BiomassTestDataset(df_unique, image_root=image_root, transform=transform)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0)

    # 3) predecir (guardamos por image_path)
    preds_map = {}  # image_path -> np.array shape (5,)
    with torch.no_grad():
        for xb, paths in test_loader:
            xb = xb.to(device)
            out = model(xb)  # (B,5)
            outputs_3 = torch.clamp(outputs_3, min=0)
            out_5 = expand_preds_3_to_5(out)
            out_5 = out_5.detach().cpu().numpy()

            for p, vec in zip(paths, out_5):
                preds_map[p] = vec

    # 4) construir submission en formato long
    #    Cada fila del test_df_long corresponde a un sample_id y un target_name
    target_idx = {name: i for i, name in enumerate(TARGET_NAMES)}

    preds = []
    for _, row in test_df_long.iterrows():
        img_path = row["image_path"]
        tname = row["target_name"]
        vec = preds_map[img_path]
        preds.append(vec[target_idx[tname]])

    submission = pd.DataFrame({
        "sample_id": test_df_long["sample_id"].values,
        "target": np.array(preds, dtype=np.float32),
    })

    submission.to_csv(out_csv_path, index=False)
    print(f"✅ Saved: {out_csv_path} | rows={len(submission)}")
    return submission


In [ ]:
test_df_long = pd.read_csv(os.path.join(DATA_DIR, "raw", "test.csv"))

submission_df = make_submission(
    model=best_model,
    device=DEVICE,
    test_df_long=test_df_long,
    image_root=os.path.join("..", "data", "resized"),
    transform=val_transforms,
    out_csv_path="submission.csv",
    batch_size=32,
)